# Sentiment Analysis Part 2

_Natural Langauge Processing Nanodegree Program_

---



## Step 5: Switching gears - RNNs

We just saw how the task of sentiment analysis can be solved via a traditional machine learning approach: BoW + a nonlinear classifier. We now switch gears and use Recurrent Neural Networks, and in particular LSTMs, to perform sentiment analysis in Keras. Conveniently, Keras has a built-in [IMDb movie reviews dataset](https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification) that we can use, with the same vocabulary size.

In [2]:
from keras.datasets import imdb  # import the built-in imdb dataset in Keras

# Set the vocabulary size
vocabulary_size = 5000

# Load in training and test data (note the difference in convention compared to scikit-learn)
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocabulary_size)
print("Loaded dataset with {} training samples, {} test samples".format(len(X_train), len(X_test)))

Using TensorFlow backend.


17465344/17464789 [==============================] - 2s 0us/step
Loaded dataset with 25000 training samples, 25000 test samples


In [3]:
# Inspect a sample review and its label
print("--- Review ---")
print(X_train[7])
print("--- Label ---")
print(y_train[7])

--- Review ---
[1, 1060, 7, 4, 2, 517, 46, 19, 6, 196, 206, 720, 15, 9, 1159, 209, 112, 1905, 1127, 95, 214, 66, 357, 18, 4, 375, 534, 5, 3496, 234, 19, 4, 172, 527, 414, 5, 753, 2, 15, 28, 2, 4, 360, 7, 4, 4857, 201, 4, 64, 155, 15, 2, 4, 3920, 9, 4, 550, 5, 13, 384, 550, 1703, 56, 8, 51, 75, 124, 80, 30, 4, 2540, 7, 2, 2, 5, 54, 15, 417, 266, 45, 184, 32, 208, 24, 87, 24, 60, 52, 21, 184, 32, 208, 14, 20, 9, 112, 2, 120, 2, 88, 12, 127, 24, 2777, 8, 312, 1827, 40, 4, 960, 2289, 305, 45, 43, 2, 753, 5, 2499, 21, 25, 28, 8, 135, 1318, 148, 1683, 2, 26, 66, 1159, 33, 211, 2, 4, 2, 1860, 507, 32, 6, 2, 947, 13, 104, 4, 465, 499, 562, 1070, 7, 3651, 54, 29, 645, 134, 2, 5, 57, 31, 1925, 14, 80, 97, 6, 2, 7, 278, 21, 1294, 558, 45, 120, 14, 280, 2, 3135, 47, 77, 3607, 180, 195, 13, 219, 4, 1691, 2, 123, 5, 103, 12, 13, 557, 6, 604, 7, 55, 392, 362, 135, 1318, 15, 16, 1190, 21, 316, 922, 74, 2310, 32, 2, 4, 172, 155, 13, 1583, 2359, 11, 4, 655, 12, 16, 243, 7, 357, 13, 43, 197, 319, 4, 2540

Notice that the label is an integer (0 for negative, 1 for positive), and the review itself is stored as a sequence of integers. These are word IDs that have been preassigned to individual words. To map them back to the original words, you can use the dictionary returned by `imdb.get_word_index()`.

In [4]:
# Map word IDs back to words
word2id = imdb.get_word_index()
id2word = {i: word for word, i in word2id.items()}
print("--- Review (with words) ---")
print([id2word.get(i, " ") for i in X_train[7]])
print("--- Label ---")
print(y_train[7])

1646592/1641221 [==============================] - 1s 0us/step
--- Review (with words) ---
['the', 'popular', 'br', 'of', 'and', 'gave', 'some', 'film', 'is', 'both', 'without', 'paul', 'for', 'it', 'bizarre', 'comedy', 'never', 'below', 'store', 'them', 'role', 'had', 'rest', 'but', 'of', 'couple', 'stories', 'to', 'brilliance', 'since', 'film', 'of', 'every', 'brilliant', 'finally', 'to', 'theme', 'and', 'for', 'one', 'and', 'of', 'hollywood', 'br', 'of', 'bumbling', 'original', 'of', 'see', '10', 'for', 'and', 'of', 'swedish', 'it', 'of', 'anyway', 'to', 'was', 'let', 'anyway', 'realized', 'she', 'in', 'when', 'bad', 'does', 'into', 'at', 'of', 'enemy', 'br', 'and', 'and', 'to', 'no', 'for', 'case', 'trying', 'if', 'around', 'an', 'times', 'his', 'him', 'his', 'which', 'very', 'not', 'around', 'an', 'times', 'as', 'on', 'it', 'never', 'and', 'show', 'and', 'most', 'that', 'end', 'his', 'nazi', 'in', 'during', 'community', 'just', 'of', 'e', 'woody', 'john', 'if', 'out', 'and', 'them

Unlike our Bag-of-Words approach, where we simply summarized the counts of each word in a document, this representation essentially retains the entire sequence of words (minus punctuation, stopwords, etc.). This is critical for RNNs to function. But it also means that now the features can be of different lengths!

#### Question: Variable length reviews

What is the maximum review length (in terms of number of words) in the training set? What is the minimum?

#### Answer:

Max = 4987 and Min = 1


### TODO: Pad sequences

In order to feed this data into your RNN, all input documents must have the same length. Let's limit the maximum review length to `max_words` by truncating longer reviews and padding shorter reviews with a null value (0). You can accomplish this easily using the [`pad_sequences()`](https://keras.io/preprocessing/sequence/#pad_sequences) function in Keras. For now, set `max_words` to 500.

In [5]:
from keras.preprocessing import sequence

# Set the maximum number of words per document (for both training and testing)
max_words = 500

# TODO: Pad sequences in X_train and X_test
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

### TODO: Design an RNN model for sentiment analysis

Build your model architecture in the code cell below. We have imported some layers from Keras that you might need but feel free to use any other layers / transformations you like.

Remember that your input is a sequence of words (technically, integer word IDs) of maximum length = `max_words`, and your output is a binary sentiment label (0 or 1).

In [9]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

# TODO: Design your model
embedding_size = 12
model = Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 12)           60000     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               45200     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 105,301
Trainable params: 105,301
Non-trainable params: 0
_________________________________________________________________
None


#### Question: Architecture and parameters

Briefly describe your neural net architecture. How many model parameters does it have that need to be trained?

#### Answer:

LSTM with Fully Connected (Dense) Layer at the end.

### TODO: Train and evaluate your model

Now you are ready to train your model. In Keras world, you first need to _compile_ your model by specifying the loss function and optimizer you want to use while training, as well as any evaluation metrics you'd like to measure. Specify the approprate parameters, including at least one metric `'accuracy'`.

In [10]:
# TODO: Compile your model, specifying a loss function, optimizer, and metrics
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Once compiled, you can kick off the training process. There are two important training parameters that you have to specify - **batch size** and **number of training epochs**, which together with your model architecture determine the total training time.

Training may take a while, so grab a cup of coffee, or better, go for a hike! If possible, consider using a GPU, as a single training run can take several hours on a CPU.

> **Tip**: You can split off a small portion of the training set to be used for validation during training. This will help monitor the training process and identify potential overfitting. You can supply a validation set to `model.fit()` using its `validation_data` parameter, or just specify `validation_split` - a fraction of the training data for Keras to set aside for this purpose (typically 5-10%). Validation metrics are evaluated once at the end of each epoch.

In [11]:
# TODO: Specify training parameters: batch size and number of epochs
batch_size = 64
num_epochs = 3

# TODO(optional): Reserve/specify some training data for validation (not to be used for training)
X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]  # first batch_size samples
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]  # rest for training

# TODO: Train your model
model.fit(X_train2, y_train2,
          validation_data=(X_valid, y_valid),
          batch_size=batch_size, epochs=num_epochs)

Train on 24936 samples, validate on 64 samples
Epoch 1/3
24936/24936 [==============================] - 493s 20ms/step - loss: 0.4888 - acc: 0.7567 - val_loss: 0.4929 - val_acc: 0.8281
Epoch 2/3
24936/24936 [==============================] - 503s 20ms/step - loss: 0.3356 - acc: 0.8621 - val_loss: 0.2887 - val_acc: 0.8281
Epoch 3/3
24936/24936 [==============================] - 513s 21ms/step - loss: 0.2870 - acc: 0.8871 - val_loss: 0.2697 - val_acc: 0.8906


In [14]:
# Save your model, so that you can quickly load it in future (and perhaps resume training)
#model_file = "rnn_model.h5"  # HDF5 file
#model.save(os.path.join(cache_dir, model_file))

# Later you can load it using keras.models.load_model()
#from keras.models import load_model
#model = load_model(os.path.join(cache_dir, model_file))

Once you have trained your model, it's time to see how well it performs on unseen test data.

In [15]:
# Evaluate your model on the test set
scores = model.evaluate(X_test, y_test, verbose=1)  # returns loss and other metrics specified in model.compile()
print("Test accuracy:", scores[1])  # scores[1] should correspond to accuracy if you passed in metrics=['accuracy']

25000/25000 [==============================] - 151s 6ms/step
Test accuracy: 0.87064


#### Question: Comparing RNNs and Traditional Methods

How well does your RNN model perform compared to the BoW + Gradient-Boosted Decision Trees?

#### Answer:

- Accuracy BoW + Gradient-Boosted Decision Trees: 73% on test
- Vanilla LSTM: 87% on test

I'm extremely surprised about the big difference. Most likely the difference stems from the nature of sentiments where word order does play an important role. In this case, if word order plays an important role, than LSTMs are the favored tool.

- In general with Deep Learning tools we should use a word embedding instead of a BoW (or Keras's hashing). We could expect even better results.
- As word order probably plays an important role, a 1D-CNN model would most likely be inferior to a LSTM model.
- An additional idea would be to 'play' with LSTMs; e.g. use a reverse input (in Keras set go_backwards to TRUE) and of course a Bidirectional LSTMs. 